In [ ]:
#from https://github.com/cfinch/Shocksolution_Examples/tree/master/PairCorrelation
def pairCorrelationFunction_3D(x, y, z, S, rMax, dr):
    """Compute the three-dimensional pair correlation function for a set of
    spherical particles contained in a cube with side length S.  This simple
    function finds reference particles such that a sphere of radius rMax drawn
    around the particle will fit entirely within the cube, eliminating the need
    to compensate for edge effects.  If no such particles exist, an error is
    returned.  Try a smaller rMax...or write some code to handle edge effects! ;)
    Arguments:
        x               an array of x positions of centers of particles
        y               an array of y positions of centers of particles
        z               an array of z positions of centers of particles
        S               length of each side of the cube in space
        rMax            outer diameter of largest spherical shell
        dr              increment for increasing radius of spherical shell
    Returns a tuple: (g, radii, interior_indices)
        g(r)            a numpy array containing the correlation function g(r)
        radii           a numpy array containing the radii of the
                        spherical shells used to compute g(r)
        reference_indices   indices of reference particles
    """
    from numpy import zeros, sqrt, where, pi, mean, arange, histogram

    # Find particles which are close enough to the cube center that a sphere of radius
    # rMax will not cross any face of the cube
    bools1 = x > rMax
    bools2 = x < (S - rMax)
    bools3 = y > rMax
    bools4 = y < (S - rMax)
    bools5 = z > rMax
    bools6 = z < (S - rMax)

    interior_indices, = where(bools1 * bools2 * bools3 * bools4 * bools5 * bools6)
    num_interior_particles = len(interior_indices)

    if num_interior_particles < 1:
        raise  RuntimeError ("No particles found for which a sphere of radius rMax\
                will lie entirely within a cube of side length S.  Decrease rMax\
                or increase the size of the cube.")

    edges = arange(0., rMax + 1.1 * dr, dr)
    num_increments = len(edges) - 1
    g = zeros([num_interior_particles, num_increments])
    radii = zeros(num_increments)
    numberDensity = len(x) / S**3

    # Compute pairwise correlation for each interior particle
    for p in range(num_interior_particles):
        index = interior_indices[p]
        d = sqrt((x[index] - x)**2 + (y[index] - y)**2 + (z[index] - z)**2)
        d[index] = 2 * rMax

        (result, bins) = histogram(d, bins=edges, normed=False)
        g[p,:] = result / numberDensity

    # Average g(r) for all interior particles and compute radii
    g_average = zeros(num_increments)
    for i in range(num_increments):
        radii[i] = (edges[i] + edges[i+1]) / 2.
        rOuter = edges[i + 1]
        rInner = edges[i]
        g_average[i] = mean(g[:, i]) / (4.0 / 3.0 * pi * (rOuter**3 - rInner**3))

    return (g_average, radii, interior_indices)



In [ ]:
from ecell4 import *
%matplotlib inline
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
from ecell4.extra import ensemble

phic=0.1
L = 1.
rm=0.005
voxel_radius = rm*1.021
D = 1.
dt = (4 * voxel_radius * voxel_radius) / (6 * D)   
print('dt',dt)
duration=dt*1e8
maxt=dt*duration
rng = GSLRandomNumberGenerator()  
rng.seed(2)
Nc=int(phic*L**3/(4*math.pi*rm**3 /3))
with species_attributes():
    C | {'D':str(D),'radius':str(voxel_radius)}
    B | {'D':str(0),'radius':str(voxel_radius)}
    T | {'D':str(0),'radius':str(voxel_radius)}

m=get_model()           
w = spatiocyte.SpatiocyteWorld(ones() * L, voxel_radius, rng)
w.bind_to(m)       
coord1 = w.position2coordinate(ones() * L * 0.5)
coord2 = w.get_neighbor(coord1, 0)
(pidB,p),suc=w.new_particle(Species('B'),w.coordinate2position(coord1))  
(pidT,p),suc=w.new_particle(Species('T'),w.coordinate2position(coord2))     
w.add_molecules(Species('C'),Nc)
sim = spatiocyte.SpatiocyteSimulator(w)
pids = [pid for pid, p in w.list_particles(Species("C"))] #particle IDs
tt= np.logspace(math.log10(dt),math.log10(duration),80)
obs = TimingTrajectoryObserver(tt,pids,False,dt)
sim.run(duration,obs)
res=np.array(obs.data())
print(res.shape)

In [ ]:
gR=[]
for pt in res.T:
    x=np.array([p[0] for p in pt])
    y=np.array([p[1] for p in pt])
    z=np.array([p[2] for p in pt])
    g_r, r, reference_indices=pairCorrelationFunction_3D(x, y, z, L,rm*2.5, rm/10)
    #print(g_r[20],r[20])    
    plt.plot(r,g_r,'.')
    gR.append(g_r[20])


In [ ]:
plt.figure()
plt.semilogx(tt[1:]/dt, gR,'o-')
plt.title('g(r=2rv), L={},phic={}'.format(L,phic))
plt.xlabel('t')
plt.ylabel('g(R)')
print('mean',np.mean(gR))
#plt.savefig("/home/chew/ecellwork/figure/gR_L{}phic{}.png".format(L,phic),format='png',dpi=100)